### Scrape text from Colombian oil contract website, found pdf links and output list of links to pdf files
### Take that list and dowloads pdfs

In [2]:
import io, os
import codecs, unicodedata, re, urlparse, urllib2
import time

In [3]:
# List of PDF URLs from Colombian website to clean
colombia_filenames_raw = 'contract_data/Contracts_Backlog/colombia_oil_contracts_raw.txt' 
# List of Clean PDF URLs from Colombian website
colombia_filenames = 'contract_data/Contracts_Backlog/colombia_oil_contracts.txt' 
# Where you want to save the PDFs
destination_folder = 'contract_data/Contracts_Backlog/'

In [4]:
f = codecs.open(colombia_filenames_raw,'rt', encoding='utf-8').readlines()
pdffiles = [filename for filename in [filename.strip() for filename in f] if filename.endswith('.pdf')]

f = codecs.open(colombia_filenames,'w',encoding='utf-8')
for filename in pdffiles:
    filename = filename.replace(' ','%20')
    f.write('http://www.anh.gov.co/Asignacion-de-areas/Contratos%20EP%20y%20TEAS%20firmados/' + filename)
    f.write('\n')
f.close()

In [5]:
existing_pdfs = [unicodedata.normalize('NFKC',f.decode('utf8')) for f in os.listdir(destination_folder) if f.lower().endswith('.pdf')]
f = list(set(codecs.open(colombia_filenames,'r', encoding='utf-8').readlines()))
print len(f)
print len(existing_pdfs)

470
474


In [6]:
def urlEncodeNonAscii(b):
    return re.sub('[\x80-\xFF]', lambda c: '%%%02x' % ord(c.group(0)), b)

def iriToUri(iri):
    parts= urlparse.urlparse(iri)
    return urlparse.urlunparse(
        part.encode('idna') if parti==1 else urlEncodeNonAscii(part.encode('utf-8'))
        for parti, part in enumerate(parts)
    )

In [7]:
def download_pdf(url,pdf_filename):
    '''
    Function downloads PDFs from url to given filename
    '''
    try:
        url = iriToUri(url)
    except:
        pass
    response = urllib2.urlopen(url)
    f = open(destination_folder + pdf_filename, 'w')
    f.write(response.read())
    f.close()
    
    return

In [8]:
for url in f:
    pdf_filename = unicodedata.normalize('NFKC',url.strip().split('/')[-1].replace("%20","_"))
    if not pdf_filename in existing_pdfs:
        print pdf_filename
        try:
            download_pdf(url.strip(),pdf_filename)
        except:
            print "Sleeping on " + pdf_filename
            time.sleep(5*60)
            try:
                download_pdf(url.strip(),pdf_filename)
            except:
                print "Could not open file: " + pdf_filename